In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # Cambia el 1 por el id de la GPU que quieras usar

In [2]:
import torch
torch.cuda.is_available(), torch.cuda.device_count(), torch.cuda.get_device_name(0)

(True, 1, 'NVIDIA RTX 4500 Ada Generation')

In [3]:
import pandas as pd
from datasets import load_dataset

dataset = load_dataset("yaful/MAGE", split="test")

SAMPLE_SIZE =1000  # MAX 60000
df_full = dataset.to_pandas()
df_sample = df_full.sample(n=SAMPLE_SIZE, random_state=42)


In [4]:
df_sample['Clase_Real'] = df_sample['label'].apply(lambda x: 'Humano' if x == 1 else 'IA')

print("\nPrimeras filas del DataFrame de muestra:")
print(df_sample[['text', 'Clase_Real', 'label']].head())


Primeras filas del DataFrame de muestra:
                                                    text Clase_Real  label
21764  Never again...never again!!' This place is ter...         IA      0
46722  put the carpet on the floor, they measure it, ...     Humano      1
49245  [substeps] You may do this process before you ...     Humano      1
30867  I believe mandatory minimum laws are unjust, c...     Humano      1
10010  Wales coach Warren Gatland has hailed Shane Wi...         IA      0


In [5]:
df_sample['text_cleaned'] = df_sample['text'].str.replace('\s+', ' ', regex=True).str.strip()

df_sample = df_sample.dropna(subset=['text_cleaned'])

df_sample.reset_index(drop=True, inplace=True)
df_sample['Texto_ID'] = df_sample.index

In [6]:
!pip install -U bitsandbytes

Defaulting to user installation because normal site-packages is not writeable


In [7]:
import torch
import torch.nn.functional as F
import pandas as pd
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoModelForMaskedLM
from tqdm import tqdm

# --- CONFIGURACIÓN DE HARDWARE ---
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DTYPE = torch.bfloat16 if DEVICE == 'cuda' else torch.float32

torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

# --- MODELOS ---
LLADA_MODEL_NAME = 'GSAI-ML/LLaDA-8B-Base'
GPT_MODEL_NAME = 'gpt2-large'
LLAMA_MODEL_NAME = "NousResearch/Llama-2-7b-hf"
BERT_MODEL_NAME = "bert-base-uncased"
ROBERTA_MODEL_NAME = "roberta-base"
GPT3_PROXY_MODEL_NAME = "EleutherAI/gpt-neo-2.7B"  


# --- INFERENCIA ---
MAX_LENGTH = 512 
BATCH_SIZE = 4           

# ----------------------------------------------------------------------------------
# 1. CARGA DE MODELOS
# ----------------------------------------------------------------------------------

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

# a) LLaDA
print("\nCargando LLaDA-8B-Base...")
tokenizer_llada = AutoTokenizer.from_pretrained(LLADA_MODEL_NAME, trust_remote_code=True)

model_llada = AutoModel.from_pretrained(
    LLADA_MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    dtype=DTYPE
).eval()

if hasattr(model_llada, "tie_weights"):
    model_llada.tie_weights()

LLADA_DEVICE = next(model_llada.parameters()).device
print("LLaDA cargado en:", LLADA_DEVICE)

# b) GPT

print("\nCargando GPT-2 Large (Proxy)...")
tokenizer_gpt = AutoTokenizer.from_pretrained(GPT_MODEL_NAME)

if tokenizer_gpt.pad_token is None:
    tokenizer_gpt.pad_token = tokenizer_gpt.eos_token

model_gpt = AutoModelForCausalLM.from_pretrained(
    GPT_MODEL_NAME,
    dtype=DTYPE
).to(DEVICE).eval()

if tokenizer_gpt.pad_token_id >= model_gpt.config.vocab_size:
    model_gpt.resize_token_embeddings(len(tokenizer_gpt))
print("GPT cargado en:", DEVICE)


# c) LLaMA

print("\nCargando LLaMA...")
tokenizer_llama = AutoTokenizer.from_pretrained(LLAMA_MODEL_NAME)

model_llama = AutoModelForCausalLM.from_pretrained(
    LLAMA_MODEL_NAME,
    quantization_config=bnb_config,
    dtype=DTYPE
).eval()

LLAMA_DEVICE = next(model_llama.parameters()).device
print("LLaMA cargado en:", LLAMA_DEVICE)



# d) BERT

print("\nCargando BERT...")
tokenizer_bert = AutoTokenizer.from_pretrained(BERT_MODEL_NAME)

model_bert = AutoModelForMaskedLM.from_pretrained(
    BERT_MODEL_NAME,
    torch_dtype=DTYPE
).to(DEVICE).eval()


BERT_DEVICE = next(model_bert.parameters()).device
print("BERT cargado en:", BERT_DEVICE)


# e) RoBERTa

print("\nCargando RoBERTa...")
tokenizer_roberta = AutoTokenizer.from_pretrained(ROBERTA_MODEL_NAME)

model_roberta = AutoModelForMaskedLM.from_pretrained(
    ROBERTA_MODEL_NAME,
    torch_dtype=DTYPE
).to(DEVICE).eval()


ROBERTA_DEVICE = next(model_roberta.parameters()).device
print("RoBERTa cargado en:", ROBERTA_DEVICE)


# f) GPT-3

print("\nCargando GPT-3 Proxy (Neo)...")
tokenizer_gpt3 = AutoTokenizer.from_pretrained(GPT3_PROXY_MODEL_NAME)

if tokenizer_gpt3.pad_token is None:
    tokenizer_gpt3.pad_token = tokenizer_gpt3.eos_token

model_gpt3 = AutoModelForCausalLM.from_pretrained(
    GPT3_PROXY_MODEL_NAME,
    quantization_config=bnb_config,
    dtype=DTYPE
).eval()

if tokenizer_gpt3.pad_token_id >= model_gpt3.config.vocab_size:
    model_gpt3.resize_token_embeddings(len(tokenizer_gpt3))

GPT3_DEVICE = next(model_gpt3.parameters()).device
print("GPT-3 Proxy cargado en:", GPT3_DEVICE)


texts = df_sample['text_cleaned']


/opt/conda/lib/python3.11/site-packages/torch/backends/__init__.py:46: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  self.setter(val)



Cargando LLaDA-8B-Base...


2026-01-08 08:19:23.936912: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-08 08:19:24.020982: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-08 08:19:25.880194: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
The model weights are not tied. Please use the `tie_weights` method before using

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

LLaDA cargado en: cuda:0

Cargando GPT-2 Large (Proxy)...
GPT cargado en: cuda

Cargando LLaMA...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LLaMA cargado en: cuda:0

Cargando BERT...


`torch_dtype` is deprecated! Use `dtype` instead!
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERT cargado en: cuda:0

Cargando RoBERTa...
RoBERTa cargado en: cuda:0

Cargando GPT-3 Proxy (Neo)...
GPT-3 Proxy cargado en: cuda:0


In [8]:
def calculate_five_metrics(logits, labels, attention_mask):
    B, T, V = logits.shape
    log_probs = F.log_softmax(logits, dim=-1)
    probs = log_probs.exp()  # Calcular probs UNA VEZ
    
    # 1. Log-prob del token ocurrido
    log_p = log_probs.gather(2, labels.unsqueeze(-1)).squeeze(-1)
    Mlog_prob = log_p
    
    # 2. Entropía
    Mentropy = -(probs * log_probs).sum(dim=-1)
    
    # 3. Máxima log-prob
    Mmax_log_prob, _ = log_probs.max(dim=-1)
    
    # 4. Rank normalizado (CORREGIDO)

    rank = (log_probs > log_p.unsqueeze(-1)).sum(dim=-1).float() + 1.0
    Mrank = rank / V
    
    # 5. Top-p (CORREGIDO)
    prob_occured = probs.gather(2, labels.unsqueeze(-1)).squeeze(-1).unsqueeze(-1)
    Mtop_p = (probs * (probs >= prob_occured)).sum(dim=-1)
    
    metrics = [Mlog_prob, Mentropy, Mmax_log_prob, Mrank, Mtop_p]
    
    seq_len = attention_mask.sum(dim=1).clamp(min=1)
    results = []
    for M in metrics:
        results.append(((M * attention_mask).sum(dim=1) / seq_len).cpu().tolist())
    
    return results

def batch_autoregressive_metrics(texts, model, tokenizer, batch_size, device):
    """
    Calcula las 5 métricas para modelos autoregresivos (LLaDA, GPT, LLaMA, GPT-3)
    """
    all_metrics = [[], [], [], [], []] # 5 listas para las 5 métricas
    
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i+batch_size].tolist()

        inputs = tokenizer(
            batch,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=MAX_LENGTH
        )
        inputs_on_device = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad(), torch.cuda.amp.autocast(dtype=DTYPE):
            outputs = model(**inputs_on_device)
            
            logits = outputs.logits[:, :-1, :]        # (B, T-1, V)
            labels = inputs_on_device["input_ids"][:, 1:] # (B, T-1)

            attention_mask = inputs_on_device["attention_mask"][:, 1:] # (B, T-1)
            
            metrics = calculate_five_metrics(logits, labels, attention_mask)
            
            for j in range(5):
                all_metrics[j].extend(metrics[j])
                
    return all_metrics


In [9]:
def calculate_five_metrics_mlm(logits, labels, attention_mask):
    """
    Calcula las 5 métricas PAWN para modelos MLM.
    Para LLaDA en modo MLM: evalúa cada posición de la secuencia.
    """
    B, T, V = logits.shape
    
    log_probs = F.log_softmax(logits, dim=-1)  # (B, T, V)
    probs = torch.exp(log_probs)
    
    # 1. Log-prob del token real
    log_prob_occured = log_probs.gather(
        2, labels.unsqueeze(-1)
    ).squeeze(-1)  # (B, T)
    
    Mlog_prob = log_prob_occured
    
    # 2. Entropía
    entropy = -(probs * log_probs).sum(dim=-1)  # (B, T)
    Mentropy = entropy
    
    # 3. Max log-prob
    Mmax_log_prob, _ = log_probs.max(dim=-1)  # (B, T)
    
    # 4. Rank (CORREGIDO - posición ordinal normalizada)
    log_prob_occured_val = log_prob_occured.unsqueeze(-1)  # (B, T, 1)
    rank = (log_probs > log_prob_occured_val).sum(dim=-1).float() + 1.0
    Mrank = rank / V  # Normalizar
    
    # 5. Top-p
    prob_occured = probs.gather(2, labels.unsqueeze(-1)).squeeze(-1).unsqueeze(-1)
    Mtop_p = (probs * (probs >= prob_occured)).sum(dim=-1)
    
    masked_metrics = [Mlog_prob, Mentropy, Mmax_log_prob, Mrank, Mtop_p]
    results = []
    sequence_lengths = attention_mask.sum(dim=1).float().clamp(min=1)  # (B,)
    
    for M in masked_metrics:
        M_masked = M * attention_mask
        M_sum_per_seq = M_masked.sum(dim=1)
        M_avg_per_seq = M_sum_per_seq / sequence_lengths
        results.append(M_avg_per_seq.cpu().tolist())
    
    return results


def batch_mlm_metrics(texts, model, tokenizer, batch_size, device):
    """
    Calcula las 5 métricas PAWN para LLaDA en modo MLM.
    LLaDA puede funcionar como MLM bidireccional.
    """
    all_metrics = [[], [], [], [], []]
    vocab_size = model.config.vocab_size
    
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i+batch_size].tolist()
        
        inputs = tokenizer(
            batch,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=MAX_LENGTH
        ).to(device)
        
        input_ids = inputs["input_ids"]
        attention_mask = inputs["attention_mask"]
        
        # CRÍTICO: Clamp de seguridad
        input_ids = input_ids.clamp(0, vocab_size - 1)
        
        with torch.no_grad(), torch.amp.autocast("cuda", dtype=DTYPE):
            # LLaDA puede retornar logits para todas las posiciones
            outputs = model(**inputs)
            
            # Si LLaDA retorna logits de forma (B, T, V), úsalos directamente
            logits = outputs.logits  # (B, T, V)
            
            # Calcular las 5 métricas
            metrics = calculate_five_metrics_mlm(logits, input_ids, attention_mask)
            
            for j in range(5):
                all_metrics[j].extend(metrics[j])
    
    return all_metrics

In [10]:
import torch
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm

def calculate_five_metrics_diffusion(logits, labels, mask_positions):
    """
    Calcula las 5 métricas PAWN solo en las posiciones ENMASCARADAS.
    Basado en la lógica de reconstrucción de Language Diffusion.
    """
    B, T, V = logits.shape
    
    # Trabajamos con log_softmax para estabilidad numérica
    log_probs = F.log_softmax(logits, dim=-1)
    probs = torch.exp(log_probs)
    
    # 1. Log-probabilidad del token real (¿Qué tan bien reconstruye el modelo el texto original?)
    log_p = log_probs.gather(2, labels.unsqueeze(-1)).squeeze(-1)
    
    # 2. Entropía (Incertidumbre del modelo en las zonas enmascaradas)
    entropy = -(probs * log_probs).sum(dim=-1)
    
    # 3. Max log-prob (Confianza máxima en la predicción)
    max_log_p, _ = log_probs.max(dim=-1)
    
    # 4. Rank normalizado (Posición ordinal del token real)
    # Calculamos cuántos tokens tienen mayor probabilidad que el token real
    rank = (log_probs > log_p.unsqueeze(-1)).sum(dim=-1).float() + 1.0
    rank_norm = rank / V
    
    # 5. Top-p (Masa de probabilidad acumulada necesaria para llegar al token real)
    # Indica qué tan "esperada" era la palabra en ese contexto
    prob_occured = probs.gather(2, labels.unsqueeze(-1)).squeeze(-1).unsqueeze(-1)
    top_p = (probs * (probs >= prob_occured)).sum(dim=-1)
    
    mask_float = mask_positions.float()
    num_masked = mask_float.sum(dim=1).clamp(min=1)
    
    results = []
    # Iteramos sobre las 5 métricas calculadas
    for M in [log_p, entropy, max_log_p, rank_norm, top_p]:
        # Filtramos para obtener el promedio SOLO de los tokens que fueron enmascarados
        # Esto es lo que mide la capacidad de "denoising" o reconstrucción.
        avg_val = (M * mask_float).sum(dim=1) / num_masked
        results.append(avg_val.cpu().tolist())
        
    return results

def batch_diffusion_metrics(texts, model, tokenizer, batch_size, device, mask_ratio=0.33, num_samples=10):
    """
    Implementa el proceso de scoring por difusión para LLaDA.
    Aumentamos el mask_ratio a 0.20 para forzar al modelo a usar más contexto.
    Reducimos num_samples a 3 para balancear velocidad y estabilidad.
    """
    all_metrics = [[] for _ in range(5)]
    
    # Identificar token de máscara correcto
    if hasattr(tokenizer, 'mask_token_id') and tokenizer.mask_token_id is not None:
        mask_id = tokenizer.mask_token_id
    else:
        # Fallback para modelos que no tienen [MASK] definido explícitamente
        mask_id = tokenizer.vocab_size - 1 

    model.eval()
    
    # Determinamos el tipo de dato para autocast (bfloat16 para GPUs modernas como RTX 4500 Ada)
    dtype = torch.bfloat16 if device == 'cuda' else torch.float32

    for i in tqdm(range(0, len(texts), batch_size), desc="LLaDA Diffusion Scoring"):
        batch_texts = texts[i:i+batch_size].tolist()
        # Acumulador para las muestras estocásticas de cada batch
        batch_accum = [[] for _ in range(5)]
        
        # Realizamos varias pasadas con diferentes máscaras para obtener un promedio robusto (Monte Carlo)
        for _ in range(num_samples):
            inputs = tokenizer(
                batch_texts, 
                return_tensors="pt", 
                padding=True, 
                truncation=True, 
                max_length=512
            ).to(device)
            
            input_ids = inputs["input_ids"]
            att_mask = inputs["attention_mask"]
            B, T = input_ids.shape
            
            # Generamos máscara aleatoria excluyendo el padding
            mask_probs = torch.full((B, T), mask_ratio, device=device) * att_mask.float()
            
            # Opcional: Evitar enmascarar tokens especiales (si el tokenizer los tiene definidos)
            if hasattr(tokenizer, 'all_special_ids'):
                for special_id in tokenizer.all_special_ids:
                    mask_probs[input_ids == special_id] = 0.0

            mask_pos = torch.bernoulli(mask_probs).bool()
            
            # Garantizar que al menos un token esté enmascarado por secuencia
            for j in range(B):
                if not mask_pos[j].any():
                    valid_indices = att_mask[j].nonzero(as_tuple=True)[0]
                    if len(valid_indices) > 0:
                        random_idx = valid_indices[torch.randint(0, len(valid_indices), (1,))]
                        mask_pos[j, random_idx] = True

            # Crear la versión "corrupta" del texto
            corrupted_ids = input_ids.clone()
            corrupted_ids[mask_pos] = mask_id
            
            with torch.no_grad(), torch.amp.autocast("cuda", dtype=dtype):
                # Predicción bidireccional: el modelo intenta adivinar los tokens en las máscaras
                outputs = model(input_ids=corrupted_ids, attention_mask=att_mask)
                
                # Extraer métricas solo de las posiciones que el modelo tuvo que reconstruir
                sample_m = calculate_five_metrics_diffusion(outputs.logits, input_ids, mask_pos)
                
                for m_idx in range(5):
                    batch_accum[m_idx].append(sample_m[m_idx])
        
        # Promediar las muestras para reducir el ruido de la selección aleatoria de máscaras
        for m_idx in range(5):
            avg_res = np.array(batch_accum[m_idx]).mean(axis=0)
            all_metrics[m_idx].extend(avg_res.tolist())
            
    return all_metrics

In [11]:
metrics_names = ['Mlog_prob', 'Mentropy', 'Mmax_log_prob', 'Mrank', 'Mtop_p']

# 1. LLaDA
print("\nCalculando 5 Métricas_LLaDA (batch)...")
results_llada = batch_diffusion_metrics(texts, model_llada, tokenizer_llada, BATCH_SIZE, LLADA_DEVICE)

for i, metric_name in enumerate(metrics_names):
    df_sample[f'{metric_name}_LLaDA'] = results_llada[i]


# 2. GPT
print("\nCalculando 5 Métricas_GPT (batch)...")
results_gpt = batch_autoregressive_metrics(texts, model_gpt, tokenizer_gpt, BATCH_SIZE, DEVICE)

for i, metric_name in enumerate(metrics_names):
    df_sample[f'{metric_name}_GPT'] = results_gpt[i]

# 3. LLaMA
print("\nCalculando 5 Métricas_LLaMA (batch)...")
results_llama = batch_autoregressive_metrics(texts, model_llama, tokenizer_llama, BATCH_SIZE, LLAMA_DEVICE)

for i, metric_name in enumerate(metrics_names):
    df_sample[f'{metric_name}_LLaMA'] = results_llama[i]

# 4. GPT-3 Proxy
print("\nCalculando 5 Métricas_GPT-3 (batch)...")
results_gpt3 = batch_autoregressive_metrics(texts, model_gpt3, tokenizer_gpt3, BATCH_SIZE, GPT3_DEVICE)

for i, metric_name in enumerate(metrics_names):
    df_sample[f'{metric_name}_GPT3'] = results_gpt3[i]

# 5. BERT
print("\nCalculando 5 Métricas_BERT (batch)...")
results_bert = batch_mlm_metrics(texts, model_bert, tokenizer_bert, BATCH_SIZE, BERT_DEVICE)

for i, metric_name in enumerate(metrics_names):
    df_sample[f'{metric_name}_BERT'] = results_bert[i]

# 6. RoBERTa
print("\nCalculando 5 Métricas_RoBERTa (batch)...")
results_roberta = batch_mlm_metrics(texts, model_roberta, tokenizer_roberta, BATCH_SIZE, ROBERTA_DEVICE)

for i, metric_name in enumerate(metrics_names):
    df_sample[f'{metric_name}_RoBERTa'] = results_roberta[i]


df_final = df_sample.dropna(subset=['Mlog_prob_LLaDA', 'Mlog_prob_GPT'], how='all')

print("\n--- Vista Previa de Scores Calculados ---")
print(df_final[['Texto_ID', 'Clase_Real', 'Mlog_prob_LLaDA', 'Mentropy_LLaDA', 'Mlog_prob_GPT', 'Mentropy_GPT']].head())

df_final.to_csv('df_final_metrics.csv', index=False)


Calculando 5 Métricas_LLaDA (batch)...


LLaDA Diffusion Scoring: 100%|██████████| 250/250 [38:37<00:00,  9.27s/it]



Calculando 5 Métricas_GPT (batch)...


/tmp/ipykernel_183264/3737245445.py:52: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast(dtype=DTYPE):
100%|██████████| 250/250 [00:45<00:00,  5.52it/s]



Calculando 5 Métricas_LLaMA (batch)...


...


Calculando 5 Métricas_BERT (batch)...


100%|██████████| 250/250 [00:10<00:00, 23.75it/s]



Calculando 5 Métricas_RoBERTa (batch)...


100%|██████████| 250/250 [00:12<00:00, 20.16it/s]



--- Vista Previa de Scores Calculados ---
   Texto_ID Clase_Real  Mlog_prob_LLaDA  Mentropy_LLaDA  Mlog_prob_GPT  \
0         0         IA        -8.280295        6.899026      -3.580119   
1         1     Humano        -6.892394        7.555557      -3.744196   
2         2     Humano        -7.125575        7.838195      -3.750089   
3         3     Humano        -7.458102        7.880184      -2.831601   
4         4         IA        -7.435955        7.474788      -1.784918   

   Mentropy_GPT  
0      3.661952  
1      4.065942  
2      3.669372  
3      2.818978  
4      2.194524  


In [12]:
import os
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
from tqdm import tqdm

SEED = 42
BATCH_SIZE = 32
EPOCHS = 200
PATIENCE = 20
LR = 1e-5
WEIGHT_DECAY = 1e-5
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_DIR = "models_mlps"
os.makedirs(MODEL_DIR, exist_ok=True)

TARGET_COL = "Clase_Real_Binaria"  
metrics_names = ['Mlog_prob', 'Mentropy', 'Mmax_log_prob', 'Mrank', 'Mtop_p']
models_list = ['LLaDA', 'GPT', 'LLaMA', 'GPT3', 'BERT', 'RoBERTa']  

torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
if DEVICE == "cuda":
    torch.cuda.manual_seed_all(SEED)

class TabularDataset(Dataset):
    def __init__(self, X, y):
        self.X = X.astype(np.float32)
        self.y = y.astype(np.float32)
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx]), torch.tensor(self.y[idx]).long()

class DeepMLP(nn.Module):
    def __init__(self, input_dim,
                 hidden_dims=(128, 64, 32),
                 dropout=0.4,
                 batchnorm=True,
                 activation=nn.ReLU,
                 final_dropout=0.25):
        super().__init__()
        layers = []
        prev = input_dim
        for h in hidden_dims:
            layers.append(nn.Linear(prev, h))
            if batchnorm:
                layers.append(nn.BatchNorm1d(h))
            layers.append(activation())
            layers.append(nn.Dropout(dropout))
            prev = h
        # final classifier
        layers.append(nn.Linear(prev, 2))
        self.net = nn.Sequential(*layers)
    def forward(self, x):
        return self.net(x)

def compute_metrics(y_true, y_pred_probs):
    y_pred = (y_pred_probs[:,1] >= 0.5).astype(int)
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    try:
        auc = roc_auc_score(y_true, y_pred_probs[:,1])
    except Exception:
        auc = float("nan")
    return {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1, "roc_auc": auc}

df = pd.read_csv(
    "df_final_metrics.csv",
    sep=",",
    engine="python",
    on_bad_lines="skip"
)
df['Clase_Real_Binaria'] = df['label']
if TARGET_COL not in df.columns:
    raise RuntimeError(f"No encuentro la columna objetivo '{TARGET_COL}' en df_final_metrics.csv. Cambia TARGET_COL al nombre correcto.")

results = {}

for model_name in models_list:
    cols = [f"{m}_{model_name}" for m in metrics_names]
    missing = [c for c in cols if c not in df.columns]
    if missing:
        print(f"[WARN] Columns missing for {model_name}: {missing}. Saltando este modelo.")
        continue

    subset = df[cols + [TARGET_COL]].dropna()
    X = subset[cols].values
    y = subset[TARGET_COL].values.astype(int)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=SEED, stratify=y
    )

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    train_ds = TabularDataset(X_train, y_train)
    test_ds = TabularDataset(X_test, y_test)
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, drop_last=False)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

    classes, counts = np.unique(y_train, return_counts=True)
    if len(counts) == 1:
        class_weights = torch.tensor([1.0, 1.0], dtype=torch.float32, device=DEVICE)
    else:
        inv = 1.0 / counts
        weights = inv / inv.sum()
        cw = np.zeros(2, dtype=np.float32)
        for cls, w in zip(classes, weights):
            cw[int(cls)] = w
        class_weights = torch.tensor(cw, dtype=torch.float32, device=DEVICE)

    input_dim = X_train.shape[1]
    hidden_dims = (1024, 512, 256, 128)   # deep y ancho
    model = DeepMLP(input_dim, hidden_dims=hidden_dims, dropout=0.4, batchnorm=True).to(DEVICE)

    optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
    criterion = nn.CrossEntropyLoss(weight=class_weights)

    best_auc = -1.0
    best_state = None
    epochs_no_improve = 0
    history = {"train_loss": [], "test_auc": []}

    for epoch in range(EPOCHS):
        model.train()
        loss_epoch = 0.0
        for xb, yb in train_loader:
            xb = xb.to(DEVICE)
            yb = yb.to(DEVICE)
            optimizer.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            optimizer.step()
            loss_epoch += loss.item() * xb.size(0)
        loss_epoch /= len(train_loader.dataset)
        history["train_loss"].append(loss_epoch)

        model.eval()
        ys = []
        yprobs = []
        with torch.no_grad():
            for xb, yb in test_loader:
                xb = xb.to(DEVICE)
                logits = model(xb)
                probs = torch.softmax(logits, dim=-1).cpu().numpy()
                yprobs.append(probs)
                ys.append(yb.numpy())
        yprobs = np.vstack(yprobs)
        ys = np.concatenate(ys)

        metrics_eval = compute_metrics(ys, yprobs)
        history["test_auc"].append(metrics_eval["roc_auc"])

        # early stopping
        if np.isfinite(metrics_eval["roc_auc"]) and metrics_eval["roc_auc"] > best_auc:
            best_auc = metrics_eval["roc_auc"]
            best_state = model.state_dict()
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epoch % 5 == 0 or epoch == EPOCHS-1:
            print(f"[{model_name}] Epoch {epoch+1}/{EPOCHS} loss={loss_epoch:.4f} test_auc={metrics_eval['roc_auc']:.4f} f1={metrics_eval['f1']:.4f}")

        if epochs_no_improve >= PATIENCE:
            print(f"[{model_name}] Early stopping en epoch {epoch+1}. Mejor AUC val: {best_auc:.4f}")
            break

    if best_state:
        model.load_state_dict(best_state)

    model.eval()
    ys = []
    yprobs = []
    with torch.no_grad():
        for xb, yb in test_loader:
            xb = xb.to(DEVICE)
            logits = model(xb)
            probs = torch.softmax(logits, dim=-1).cpu().numpy()
            yprobs.append(probs)
            ys.append(yb.numpy())
    yprobs = np.vstack(yprobs)
    ys = np.concatenate(ys)
    final_metrics = compute_metrics(ys, yprobs)
    print(f"\n>>> RESULTADOS FINALES para {model_name}: {final_metrics}\n")

    torch.save({
        "model_state": model.state_dict(),
        "scaler_mean": scaler.mean_,
        "scaler_scale": scaler.scale_,
        "cols": cols,
        "metrics": final_metrics,
        "history": history
    }, os.path.join(MODEL_DIR, f"mlp_{model_name}.pt"))

    results[model_name] = final_metrics

df_results = pd.DataFrame(results).T
print("===== Resumen comparativo =====")
print(df_results)
best_model = df_results['roc_auc'].idxmax()
print(f"\nMejor modelo según ROC-AUC: {best_model} -> {df_results.loc[best_model].to_dict()}")

# guardado de resultados
df_results.to_csv(os.path.join(MODEL_DIR, "comparison_results.csv"))
print(f"Modelos y resultados guardados en '{MODEL_DIR}/'")

[LLaDA] Epoch 1/200 loss=0.7496 test_auc=0.5412 f1=0.0200
[LLaDA] Epoch 6/200 loss=0.7386 test_auc=0.6101 f1=0.1552
[LLaDA] Epoch 11/200 loss=0.7165 test_auc=0.6345 f1=0.3546
[LLaDA] Epoch 16/200 loss=0.7002 test_auc=0.6524 f1=0.4189
[LLaDA] Epoch 21/200 loss=0.7090 test_auc=0.6507 f1=0.4387
[LLaDA] Epoch 26/200 loss=0.6956 test_auc=0.6514 f1=0.5537
[LLaDA] Epoch 31/200 loss=0.6882 test_auc=0.6574 f1=0.5549
[LLaDA] Epoch 36/200 loss=0.7035 test_auc=0.6574 f1=0.5934
[LLaDA] Epoch 41/200 loss=0.7192 test_auc=0.6541 f1=0.6011
[LLaDA] Epoch 46/200 loss=0.7242 test_auc=0.6567 f1=0.6162
[LLaDA] Epoch 51/200 loss=0.7154 test_auc=0.6593 f1=0.6162
[LLaDA] Epoch 56/200 loss=0.7010 test_auc=0.6565 f1=0.6096
[LLaDA] Epoch 61/200 loss=0.6778 test_auc=0.6618 f1=0.5907
[LLaDA] Early stopping en epoch 62. Mejor AUC val: 0.6638

>>> RESULTADOS FINALES para LLaDA: {'accuracy': 0.61, 'precision': 0.6041666666666666, 'recall': 0.5918367346938775, 'f1': 0.5979381443298969, 'roc_auc': 0.6633653461384553}

[

In [13]:
# ============================================================
# GRID SEARCH LLaDA (SILENCIADO)
# ============================================================

import pandas as pd

mask_ratios = [ 0.25,0.275, 0.3,0.325, 0.35,0.375, 0.4,0.425]
num_samples_list = [ 7,10]

grid_results = []

texts = df["text"]
labels = df[TARGET_COL].values

for mask_ratio in mask_ratios:
    for num_samples in num_samples_list:

        llada_metrics = batch_diffusion_metrics(
            texts=texts,
            model=model_llada,
            tokenizer=tokenizer_llada,
            batch_size=4,
            device=DEVICE,
            mask_ratio=mask_ratio,
            num_samples=num_samples
        )

        llada_df = pd.DataFrame(
            np.column_stack(llada_metrics),
            columns=[f"{m}_LLaDA" for m in metrics_names]
        )
        llada_df[TARGET_COL] = labels
        llada_df.dropna(inplace=True)

        X = llada_df.drop(columns=[TARGET_COL]).values
        y = llada_df[TARGET_COL].values.astype(int)

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, stratify=y, random_state=SEED
        )

        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        train_loader = DataLoader(
            TabularDataset(X_train, y_train),
            batch_size=BATCH_SIZE,
            shuffle=True
        )
        test_loader = DataLoader(
            TabularDataset(X_test, y_test),
            batch_size=BATCH_SIZE
        )

        model_mlp = DeepMLP(
            input_dim=X_train.shape[1],
            hidden_dims=(128, 64, 32),
            dropout=0.4
        ).to(DEVICE)

        optimizer = torch.optim.AdamW(
            model_mlp.parameters(),
            lr=LR,
            weight_decay=WEIGHT_DECAY
        )
        criterion = nn.CrossEntropyLoss()

        best_auc = -1
        patience = 0

        for _ in range(100):
            model_mlp.train()
            for xb, yb in train_loader:
                xb, yb = xb.to(DEVICE), yb.to(DEVICE)
                optimizer.zero_grad()
                loss = criterion(model_mlp(xb), yb)
                loss.backward()
                optimizer.step()

            model_mlp.eval()
            ys, yprobs = [], []
            with torch.no_grad():
                for xb, yb in test_loader:
                    probs = torch.softmax(model_mlp(xb.to(DEVICE)), dim=-1)
                    yprobs.append(probs[:,1].cpu())
                    ys.append(yb)

            auc = roc_auc_score(
                torch.cat(ys).numpy(),
                torch.cat(yprobs).numpy()
            )

            if auc > best_auc:
                best_auc = auc
                patience = 0
            else:
                patience += 1
                if patience >= 10:
                    break

        grid_results.append({
            "mask_ratio": mask_ratio,
            "num_samples": num_samples,
            "roc_auc": best_auc
        })

        del model_mlp
        torch.cuda.empty_cache()


LLaDA Diffusion Scoring: 100%|██████████| 250/250 [28:16<00:00,  6.79s/it]


In [14]:
df_grid = pd.DataFrame(grid_results).sort_values("roc_auc", ascending=False)
print(df_grid.head())
print("\n🏆 Mejores parámetros:")
print(df_grid.iloc[0])


    mask_ratio  num_samples   roc_auc
13       0.400           10  0.664366
6        0.325            7  0.663365
9        0.350           10  0.649960
3        0.275           10  0.636555
15       0.425           10  0.634754

🏆 Mejores parámetros:
mask_ratio      0.400000
num_samples    10.000000
roc_auc         0.664366
Name: 13, dtype: float64


In [15]:
df_grid

,mask_ratio,num_samples,roc_auc
13,0.400,10,0.664366
6,0.325,7,0.663365
9,0.350,10,0.649960
3,0.275,10,0.636555
15,0.425,10,0.634754
8,0.350,7,0.626551
11,0.375,10,0.626150
4,0.300,7,0.616046
7,0.325,10,0.615646
12,0.400,7,0.601941
